In [48]:
# install api interface at https://github.com/cwendt94/ff-espn-api/

import requests

# get hardcoded cookies from browser


# helper function from ff-espn-api
def checkRequestStatus(status: int) -> None:
    if 500 <= status <= 503:
            raise Exception(status)
    if status == 401:
        raise Exception("Access Denied")

    elif status == 404:
        raise Exception("Invalid League")

    elif status != 200:
        raise Exception('Unknown %s Error' % status)


year = 2020
league_id = 83174673


# fetch league. need some data from cookies to fill in here
swid = '{9F7C3455-C43D-42A6-9D1A-AEB707CB5F0B}'
espn_s2 = 'AEAy3mkAv%2FsnK2YbgEVRFKpOw6%2ByQVeXH5BDBOlzgAiBg646ZtRuneInbY1z9QYrt1Ws8r6Cc8pS0g%2BVi7HmU0b%2F0lxgkP9csOvk9S01pusHVMJOZ9GfMZblZ01U9NjD46R2N%2BI21guim1Lga4TfVKqE6rCLMJvQszv5ZKdFgjngR98gxiClp9R3ZxzJ7cTLJufuir4JGnzKZ9RwOCL1%2BOaFy8qFEGwEVNGkuMY4Hj4zSR%2BL9a5fatMYEWNXvk4muhbgckHixSZiqvSnL6pyniu98SRFEpigMIkWgSJ%2FTfNvCg%3D%3D'
cookies = {
    'espn_s2': espn_s2,
    'SWID': swid
}
ENDPOINT = "https://fantasy.espn.com/apis/v3/games/FFL/seasons/" + str(year) + "/segments/0/leagues/" + str(league_id)
params = ''

# test request
r = requests.get(ENDPOINT, params=params, cookies=cookies)
status = r.status_code
checkRequestStatus(status)
data = r.json()
print(data)

{'gameId': 1, 'id': 83174673, 'members': [{'displayName': 'espn04014086', 'id': '{23752840-5024-4BC3-B528-4050243BC3EE}', 'isLeagueManager': False}, {'displayName': 'YungFly', 'id': '{2A52739B-A4D2-440F-B78D-C1282294B36E}', 'isLeagueManager': False}, {'displayName': 'Ball, call, install', 'id': '{5E1655AA-B7E8-4112-8208-4024E2B806B4}', 'isLeagueManager': False}, {'displayName': 'espn57414091', 'id': '{86F13096-2F1D-44BC-B130-962F1DA4BCEA}', 'isLeagueManager': False}, {'displayName': 'broncoz16', 'id': '{89BABC79-3E89-4484-B7E0-91FE30264A36}', 'isLeagueManager': False}, {'displayName': 'typhe12', 'id': '{8BCE107A-C396-4216-A165-204DCBB7ACED}', 'isLeagueManager': False}, {'displayName': 'ESPNFAN6778113520', 'id': '{9DAF6913-3970-416F-BECB-D0DE77788ADB}', 'isLeagueManager': False}, {'displayName': 'shottysnipesonthepit', 'id': '{9F7C3455-C43D-42A6-9D1A-AEB707CB5F0B}', 'isLeagueManager': False}, {'displayName': 'JackMichaels55', 'id': '{AED6F6FD-B3C1-49EC-9329-C5E971023ED7}', 'isLeagueMana

# Pull data from ESPN fantasy league and save it in a tabular format

In [49]:
import numpy as np

# fetch players, get scoring data
def get_player_data(week):
    params = {
        'scoringPeriodId': week
    }
    endpoint = 'https://fantasy.espn.com/apis/v3/games/ffl/seasons/' + str(year) + '/segments/0/leagues/' + str(league_id) + '?view=mMatchup&view=mMatchupScore'
    r = requests.get(endpoint, params=params, cookies=cookies)
    status = r.status_code
    checkRequestStatus(status)
    d = r.json()
    return d

# get player scoring data for all weeks
import pandas as pd

slotcodes = {
    0 : 'QB', 2 : 'RB', 4 : 'WR',
    6 : 'TE', 16: 'Def', 17: 'K',
    20: 'Bench', 21: 'IR', 23: 'Flex'
}

# holder variable
data = []

# iterate teams
for week in range(1, 14):

    # get player data for that scoring period
    d = get_player_data(week)

    for tm in d['teams']:
        tmid = tm['id']

        # iterate players on teams
        for p in tm['roster']['entries']:
            name = p['playerPoolEntry']['player']['fullName']
            slot = p['lineupSlotId']
            pos  = slotcodes[slot]

            # injured status (need try/exc bc of D/ST)
            inj = 'NA'
            try:
                inj = p['playerPoolEntry']['player']['injuryStatus']
            except:
                pass

            # grab projected/actual points
            proj, act = None, None
            for stat in p['playerPoolEntry']['player']['stats']:
                if stat['scoringPeriodId'] != week:
                    continue
                if stat['statSourceId'] == 0:
                    act = stat['appliedTotal']
                elif stat['statSourceId'] == 1:
                    proj = stat['appliedTotal']

            data.append([
                week, tmid, name, slot, pos, inj, proj, act
            ])
    print('Week {} complete.'.format(week))

data = pd.DataFrame(data, columns=['Week', 'Team', 'Player', 'Slot', 'Pos', 'Status', 'Proj', 'Actual'])
# data.to_csv('20201209_data.csv')


Week 1 complete.
Week 2 complete.
Week 3 complete.
Week 4 complete.
Week 5 complete.
Week 6 complete.
Week 7 complete.
Week 8 complete.
Week 9 complete.
Week 10 complete.
Week 11 complete.
Week 12 complete.
Week 13 complete.


# Utility functions for data analysis

In [9]:
import json


def load_data(year='2020'):
    if year == '2020':
        data_fname = '20201209_2020_data.csv'
    if year == '2019':
        data_fname = '20201211_2019_data.csv'

    return pd.read_csv(data_fname)

# convenience functions
def get_player_performance_rank(player, week, only_starters=True, year='2020'):
    """ function that evaluates the positional ranking of a player among players owned (or just starters) that week
    I think this metric has better insight into how good a player is. random waiver players or bench players who
    pop off are better thought of as noise, assuming we are rational managers """

    # load data
    data = load_data(year)
    if only_starters:
        data = data[data['Pos'] != 'Bench']
    
    # just work with data for that week
    week_data = data[data['Week'] == week]

    # get position of that player
    pos = week_data[week_data['Player'] == player]['Pos']

    # return nan if player didn't start that week
    # and the person had "only_starters" selected
    if pos.empty:
        print("Warning: player {} not in dataset for week {}. Returning NaN.".format(player, week))
        return np.NaN   

    # get scores of players of that position
    pos_data = week_data[week_data['Pos'] == pos.unique()[0]]

    # get scores and sort descending
    scores = pos_data['Actual'].unique()
    scores.sort()
    scores = scores[::-1]

    # get where player score is in rank of scores
    player_score = pos_data[pos_data['Player']==player]['Actual'].tolist()[0]
    rank = np.where(scores == player_score)[0][0]

    # increment cause zero indexing
    rank += 1

    return rank


def get_player_score(player, week, projected=False, year='2020'):
    """ Convenience function to get score of a player given data and week
    """

    data = load_data(year)
    data = data[data['Week'] == week]

    # get data for that player
    player_data = data[data['Player'] == player]

    # return nan if player didn't score in that dataset for whatever reason
    if player_data.empty:
        print("Warning: no player score for {} recorded for week {}.".format(player, week))
        return np.NaN

    # return score
    if projected:
        target = 'Proj'
    else:
        target = 'Actual'
    player_score = player_data[target].tolist()[0]

    # if player was on bye, score will be zero but there's no other indication
    if np.isnan(player_score):
        #print('Warning: getting score from player {} on bye week {}, returning 0'.format(player, week))
        player_score = 0

    return player_score


def was_player_started(player, week, year='2020'):

    # just work with data for that week
    data = load_data(year)
    data = data[data['Week'] == week]
    player_data = data[data['Player'] == player]

    # if player wasn't in dataset, he was in waivers
    if player_data.empty:
        return False

    # if player was in dataset as bench, he was on bench
    if player_data['Pos'] == 'Bench':
        return False

    # if player was on IR, he's basically on the bench
    if player_data['Pos'] == 'IR':
        return False

    # if not the above, then ya
    return True


def was_player_on_bench(player, week, year='2020'):

    # just work with data for that week
    data = load_data(year)
    data = data[data['Week'] == week]
    player_data = data[data['Player'] == player]

    # if player wasn't in dataset, he was in waivers
    if player_data.empty:
        return False

    # if player was in dataset as bench, he was on bench
    if player_data['Pos'] == 'Bench':
        return True

    # if player was on IR, he's basically on the bench
    if player_data['Pos'] == 'IR':
        return True
    
    # otherwise player started
    return False


def was_player_on_waivers(player, week, year='2020'):

    # just work with data for that week
    data = load_data(year)
    data = data[data['Week'] == week]
    player_data = data[data['Player'] == player]

    # if player wasn't in dataset, he was in waivers
    if player_data.empty:
        return True
    else:
        return False


def get_team_id(nickname=None, year='2020'):
    """ get the team id given a nickname for convenience """

    if year == '2020':
        nicknames = {
            'Tyler': 1,
            'Ray': 4,
            'Blount': 8,
            'Jack': 7,
            'Poogz': 10,
            'Brian': 3,
            'Mitch': 2,
            'D\'vonne': 5,
            'Sam': 6,
            'Hogz': 9
        }
    elif year == '2019':
        nicknames = {
            'Tyler': 1,
            'Ray': 4,
            'Blount': 8,
            'Jack': 7,
            'Brian': 3,
            'Mitch': 2,
            'D\'vonne': 5,
            'Sam': 6
        }

    # return nickname list if none argument
    if nickname is None:
        return nicknames

    if nickname not in nicknames.keys():
        print('Warning: nickname {} not recognized in list {}'.format(nickname, nicknames))
        return -1
    
    return nicknames[nickname]

def get_team_nickname(teamid, year='2020'):

    nicknames = get_team_id(None, year=year)

    # flip key-value pairs
    ids = dict([(value, key) for key, value in nicknames.items()]) 

    # return
    return ids[teamid]


#def get_team_nicknames_list(year='2020'):
#    nicknames = get_team_id(nickname=None, year=year).keys()
#    return nicknames

def get_team_starters(team, week, pos=None, year='2020'):
    """ get players a team started that week. team can be nickname or id. optional argument to get players of a certain position"""
    
    if type(team) == str:
        team_ndx = get_team_id_from_nickname(team, year)
    else:
        team_ndx = team

    # load data and just get data for that week/team
    data = load_data(year)
    data = data[data['Week'] == week]
    data = data[data['Team'] == team_ndx]

    # get players not on bench or IR
    data = data[data['Pos'] != 'Bench']
    starters = data[data['Pos'] != 'IR']

    if pos is not None:
        starters = starters[starters['Pos'] == pos]
        if starters.empty:
            print('Warning: position {} not found in starters for team {} week {}.'.format(pos, team, week))
            return []
    
    return starters['Player'].tolist()


#def get_player_position(player, year='2020'):
#    """ return position of player. heavy assumption player pos doesn't change """
#
#    data = load_data(year)
#    pos = data[data['Player'] == player]['Pos'].tolist()[0]
#    return pos


def get_team_color(team, year='2020'):
    if type(team) == str:
        team_ndx = get_team_id_from_nickname(team, year)
    else:
        team_ndx = team

    team_colors = np.array(plt.get_cmap('tab10').colors)
    return team_colors[team_ndx-1,:]


def get_number_of_position_slots(pos=None, year='2020'):

    if year == '2020':
        slots = 0
        if pos is None:
            slots = 10
        elif pos == 'QB':
            slots = 1
        elif pos == 'RB':
            slots = 2
        elif pos == 'WR':
            slots = 3
        elif pos == 'K':
            slots = 1
        elif pos == 'Def':
            slots = 1
        elif pos == 'TE':
            slots = 1
        else:
            print('Position {} not recognized.'.format(pos))
            slots = -1
    
    elif year == '2019':
        slots = 0
        if pos is None:
            slots = 10
        elif pos == 'QB':
            slots = 1
        elif pos == 'RB':
            slots = 2
        elif pos == 'WR':
            slots = 3
        elif pos == 'K':
            slots = 1
        elif pos == 'Def':
            slots = 1
        elif pos == 'TE':
            slots = 1
        else:
            print('Position {} not recognized.'.format(pos))
            slots = -1
    
    return slots
    

def get_num_players_started(team, pos=None, year='2020'):
    """ function that returns the total number of different players started, optionally in a position """

    if type(team) == str:
        team_ndx = get_team_id_from_nickname(team, year)
    else:
        team_ndx = team

    # load team data
    data = load_data(year)
    data = data[data['Team'] == team_ndx]

    # get players not on bench or IR
    data = data[data['Pos'] != 'Bench']
    data = data[data['Pos'] != 'IR']

    if pos is None:
        num_players_started = len(data['Player'].unique())
    else:
        data = data[data['Pos'] == pos]
        num_players_started = len(data['Player'].unique())
    
    return num_players_started




In [12]:
# some convenience functions that require espn_api (https://github.com/cwendt94/espn-api)
from espn_api.football import League
from espn_api.football import Player

# hardcoded
year = 2020
league_id = 83174673
swid = '{9F7C3455-C43D-42A6-9D1A-AEB707CB5F0B}'
espn_s2 = 'AEAy3mkAv%2FsnK2YbgEVRFKpOw6%2ByQVeXH5BDBOlzgAiBg646ZtRuneInbY1z9QYrt1Ws8r6Cc8pS0g%2BVi7HmU0b%2F0lxgkP9csOvk9S01pusHVMJOZ9GfMZblZ01U9NjD46R2N%2BI21guim1Lga4TfVKqE6rCLMJvQszv5ZKdFgjngR98gxiClp9R3ZxzJ7cTLJufuir4JGnzKZ9RwOCL1%2BOaFy8qFEGwEVNGkuMY4Hj4zSR%2BL9a5fatMYEWNXvk4muhbgckHixSZiqvSnL6pyniu98SRFEpigMIkWgSJ%2FTfNvCg%3D%3D'

# instantiate league object
league = League(league_id=league_id, year=year, espn_s2=espn_s2, swid=swid)

# return player ID given name
def get_player_id(player_name):

    pid = league.player_map[player_name]

    return pid

# get stats on player performance
def get_player_object(player, year=2020):

    if type(player) == int:
        pid = player
    else:
        pid = get_player_id(player_name)

    # instantiate player object given player ID
    params = { 'view': 'kona_playercard' }
    filters = {'players':{'filterIds':{'value':[pid]}, 'filterStatsForTopScoringPeriodIds':{'value':16, "additionalValue":["00{}".format(year), "10{}".format(year)]}}}
    headers = {'x-fantasy-filter': json.dumps(filters)}
    data = league.espn_request.league_get(params=params, headers=headers)
    p = Player(data['players'][0], year)

    return p

def get_player_position(player, year):

    pid = get_player_id(player_name)
    p = get_player_object(pid)
    return p.position    

# Data analysis

### plot formatting

In [10]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import json
%matplotlib qt

# some overall decoration settings
sns.set_style('darkgrid')
plt.xkcd()
weeks = list(range(1,14))
linewidth=3


### Plot team score or individual position performance

In [7]:
# plot sum player score by week for each position. pos=None means whole team
year = '2020'
pos = 'QB'

# constants
num_teams = len(get_team_id(None, year=year).keys())
num_weeks = len(weeks)
num_pos_slots = get_number_of_position_slots(pos, year=year)

# initialize output data structure
data_array = np.empty(shape=(num_teams, num_weeks, num_pos_slots))

# iterate teams and weeks
for t in range(num_teams):
    for w in range(num_weeks):

        # iterate players in position, +1 for 1-indexing when looking up team/week score
        tid = t + 1
        wid = w + 1
        players = get_team_starters(tid, wid, pos=pos, year=year)
        for p in range(len(players)):
            rank = get_player_score(players[p], wid, year=year)
            data_array[t, w, p] = rank

# get average
avg = np.nansum(np.nanmean(data_array, axis=0), axis=1)

# plot each team
for t in range(num_teams):

    to_plot = np.nansum(data_array[t,:,:], axis=1)

    # get number of weeks above average
    avg_relative_to_avg = np.round(np.nanmean(to_plot - avg), decimals=1)
    tid = t+1
    color = get_team_color(tid, year=year)
    nn = get_team_nickname(tid, year=year)
    num_p = get_num_players_started(tid, pos, year=year)
    plt.plot(weeks, to_plot, color=color, linewidth=linewidth, label=nn + '(' + str(num_p) + 'Δ{})'.format(avg_relative_to_avg))


# plot average
plt.plot(weeks, avg, color='k', linewidth=linewidth, label='average')

# decorate
#plt.legend(ncol=4, loc='lower center')
plt.legend(ncol=4, loc='upper center')
plt.xticks(weeks)
plt.ylabel('Total scoring by week')
plt.xlabel('Week')
plt.title('Scoring by team')
plt.title('Scoring of {} by team'.format(pos))
#plt.ylim([60, 240])
plt.ylim([0, 50])


(0.0, 50.0)

In [73]:
# instantiate league object
num_teams = 8
year = 2019
league_2019 = League(league_id=league_id, year=2019, espn_s2=espn_s2, swid=swid)
league_2020

# get scoring from draft players
draft = league.draft

# make output data structure 
rounds = 15
arr = np.zeros((num_teams, num_teams*rounds, num_weeks))

for picknum in range(num_teams*rounds):

    # get stats from that player
    pick = draft[picknum]
    player_id = pick.playerId
    player = get_player_object(player_id, year=year)
    stats = player.stats

    # get info on team drafting
    team = pick.team
    tid = team.team_id

    # iterate weeks and slot in data. skips total
    weeks_eligible = stats.keys()
    for w in weeks_eligible:
        if w == 0 or w > 13:
            continue
        score = stats[w]['points']
        
        # store in output data structure
        round_num = pick.round_num
        arr[tid-1, round_num-1, w-1] = score
    
    if picknum % 20 == 0:
        print('Pick {}, player: {}, scoring: {}'.format(picknum, player.name, arr[tid-1, round_num-1, :]))


Pick 0, player: Christian McCaffrey, scoring: [42.9  7.3 27.8 33.9 47.7 21.7  0.  27.5 37.6 26.1 30.1 34.3 17.2]
Pick 20, player: Adam Thielen, scoring: [13.3 12.5 20.6  2.6 32.  17.7  9.5  0.   0.   0.   0.   0.   0. ]
Pick 40, player: Kenny Golladay, scoring: [14.2 25.7  3.7 23.7  0.  17.1  3.1 28.3 23.2 14.7  4.4 10.1 25.8]
Pick 60, player: Jarvis Landry, scoring: [11.7  6.2  9.2 24.7 11.5  6.6  0.  11.5 17.1 24.7 14.3 36.8 13.6]
Pick 80, player: Lamar Miller, scoring: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Pick 100, player: Jaguars D/ST, scoring: [-8. 10. 15.  1.  1.  7. 20. 18.  2.  0.  5. -3.  6.]


In [75]:
# plot top n draft player performance by team
rstart = 0
rend = 5
avg = arr[:,rstart:rend,:].mean(axis=0).sum(axis=0)
for t in range(num_teams):

    # plot each team with appropriate styling
    tid = t + 1 
    nn = get_team_nickname(tid)
    color = get_team_color(tid)
    to_plot = arr[t,rstart:rend,:].sum(axis=0)
    
    # get average
    avg_relative_to_avg = np.round(np.nanmean(to_plot - avg), decimals=1)

    # plot
    plt.plot(weeks, to_plot, linewidth=linewidth, color=color, label=nn + '(Δ{})'.format(avg_relative_to_avg))

# plot average
plt.plot(weeks, avg, linewidth=linewidth, color='k', label='avg')

# decorate
plt.legend(ncol=4, loc='upper center')
plt.xticks(weeks)
plt.ylabel('Total scoring by week')
plt.xlabel('Week')
plt.title('Sum of scoring of rounds {}:{} draft picks by team'.format(rstart+1, rend+1, pos))
plt.ylim([0, 160])


(0.0, 160.0)

In [49]:
# instantiate league object
num_teams = 8
league_2019 = League(league_id=league_id, year=2019, espn_s2=espn_s2, swid=swid)
league_2020 = League(league_id=league_id, year=2029, espn_s2=espn_s2, swid=swid)


array([78.978, 77.154, 71.886, 68.054, 59.526, 49.894, 53.38 , 47.736,
       52.058, 41.446, 62.072, 56.098, 52.74 ])

In [22]:
# percent of total team score accounted for by drafted players
# or percent of total team score of players that a person drafted


Player(Travis Kelce)